In [1]:
import sys
sys.path.append('/Users/ianleefmans/Desktop/stock-sentiment/utils')
from data import Database, ScrapeWSB, Stock
from datahelper import PostDataset, CommentDataset, get_indices
from torch.utils.data import DataLoader
from inference import RunInference
from models import FineTuneBaseModel, FineTuneClassifier
%cd /Users/ianleefmans/Desktop/stock-sentiment

/Users/ianleefmans/Desktop/stock-sentiment


In [2]:
import transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [82]:

db = Database()
db.use_database('DB1')

print(db.query('show tables;'))

connection established
[(datetime.datetime(2021, 6, 14, 18, 22, 44),)]
[('COMMENTS',), ('POSTS',), ('STOCKS',)]


In [7]:
db.query("SELECT * FROM STOCKS WHERE STOCK_ID='AMC' AND LAST_SCRAPED >= DATE_SUB(NOW(),INTERVAL 55 MINUTE)")

[('AMC', datetime.datetime(2021, 6, 6, 22, 15, 44))]

In [16]:
db.query("SELECT POST_ID FROM POSTS WHERE STOCK_ID='{}' AND LAST_SCRAPED >= DATE_SUB((SELECT LAST_SCRAPED FROM STOCKS WHERE STOCK_ID = '{}'), INTERVAL {} HOUR);".format('BB', 'BB', 6))



[('nx2ihx',),
 ('nxcb5h',),
 ('nxp5r6',),
 ('nxtjj4',),
 ('nxv9jr',),
 ('nxx9sz',),
 ('ny01wx',),
 ('nz6tgd',),
 ('nzf0mj',),
 ('nzjcfg',)]

In [86]:
db.query("SELECT POST_ID FROM POSTS WHERE STOCK_ID='{}' AND LAST_SCRAPED >= DATE_SUB((SELECT LAST_SCRAPED FROM STOCKS WHERE STOCK_ID = '{}'),INTERVAL {} HOUR);".format('AAPL','AAPL',6))



[('nv0h06',)]

In [85]:
db.query("SELECT LAST_SCRAPED FROM POSTS WHERE LAST_SCRAPED >= DATE_SUB((SELECT LAST_SCRAPED FROM STOCKS WHERE STOCK_ID = 'AAPL'),INTERVAL 6 HOUR);")
         
         
         


[(datetime.datetime(2021, 6, 14, 18, 21, 27),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 43),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 43),),
 (datetime.datetime(2021, 6, 14, 14, 27, 43),),
 (datetime.datetime(2021, 6, 14, 14, 27, 43),)]

In [89]:
db.query("SELECT POST_ID, LAST_SCRAPED FROM POSTS WHERE STOCK_ID='AAPL'")

[('mzzf2q', datetime.datetime(2021, 6, 7, 1, 38, 7)),
 ('n0ivmx', datetime.datetime(2021, 6, 7, 1, 38, 7)),
 ('n1kq5z', datetime.datetime(2021, 6, 7, 1, 38, 6)),
 ('n226xb', datetime.datetime(2021, 6, 7, 1, 38, 6)),
 ('n2guu3', datetime.datetime(2021, 6, 7, 1, 38, 6)),
 ('n3m6m0', datetime.datetime(2021, 6, 7, 1, 38, 5)),
 ('n51zx5', datetime.datetime(2021, 6, 7, 1, 38, 7)),
 ('n5sglk', datetime.datetime(2021, 6, 7, 1, 38, 7)),
 ('n6991f', datetime.datetime(2021, 6, 7, 1, 38, 7)),
 ('n6fa9f', datetime.datetime(2021, 6, 7, 1, 38, 5)),
 ('n6k7wf', datetime.datetime(2021, 6, 7, 1, 38, 6)),
 ('n6m3qj', datetime.datetime(2021, 6, 7, 1, 38, 5)),
 ('n799ne', datetime.datetime(2021, 6, 7, 1, 38, 6)),
 ('n7fe53', datetime.datetime(2021, 6, 7, 1, 38, 6)),
 ('n8coni', datetime.datetime(2021, 6, 7, 1, 38, 6)),
 ('n8mv7h', datetime.datetime(2021, 6, 7, 1, 38, 5)),
 ('n90iyg', datetime.datetime(2021, 6, 7, 1, 38, 5)),
 ('n93o6x', datetime.datetime(2021, 6, 7, 1, 38, 5)),
 ('na8f8j', datetime.datetim

In [45]:
indices = db.query("SELECT POST_ID FROM POSTS WHERE StOCK_ID='{}';".format("GME"))

In [4]:
db = Database()
db.use_database('DB1')
#db.drop_table("LABELED_POSTS")
#db.drop_table("LABELED_COMMENTS")

db.initialize_tables()

scrapewsb = ScrapeWSB('GME', 11, 2)

df = scrapewsb.scrape()
scrapewsb.convert(df)
print("DONE")

connection established
[(datetime.datetime(2021, 6, 6, 17, 23, 59),)]
connection established
[(datetime.datetime(2021, 6, 6, 17, 24),)]


  0%|          | 0/11 [00:00<?, ?it/s]

connection established
[(datetime.datetime(2021, 6, 6, 17, 24, 3),)]


100%|██████████| 11/11 [01:08<00:00,  6.24s/it]

DONE


In [4]:
db = Database()
db.use_database('DB1')

connection established
[(datetime.datetime(2021, 5, 30, 0, 20, 6),)]


In [3]:
db.drop_table('COMMENTS')
db.drop_table("POSTS")
db.drop_table("STOCKS")

In [2]:
indices = get_indices("BB")

connection established
[(datetime.datetime(2021, 6, 15, 21, 47, 15),)]


In [3]:
post_train = PostDataset(512, indices['post_train'])



connection established
[(datetime.datetime(2021, 6, 15, 21, 47, 18),)]


In [4]:
post_trainloader = DataLoader(
    dataset=post_train, 
    batch_size=3, 
    num_workers=0,
    shuffle=True
    )

In [5]:
sample = iter(post_trainloader).next()

/Users/ianleefmans/opt/anaconda3/envs/StockSentiment/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [6]:
sample


{'post': ['(Part 2) AMC & GME Price Action Comparison before their squeeze',
  '❗“The Monkey Fool”❗ 🤡: A Parody on How Bullsh!t FUD Campaigns Are Made for GameStop 😹 A Professional',
  'AMC YOLO LFG!'],
 'post_input_ids': tensor([[  101,   113,  4539,  ...,     0,     0,     0],
         [  101,   100,   789,  ...,     0,     0,     0],
         [  101, 24810,   162,  ...,     0,     0,     0]]),
 'post_attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'target': tensor([0, 0, 1])}

In [7]:
model = FineTuneClassifier()

out = model(input_ids=sample['post_input_ids'], attention_masks=sample['post_attention_mask'])

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [8]:
out

tensor([[0.5016, 0.4984],
        [0.5107, 0.4893],
        [0.4868, 0.5132]], grad_fn=<SoftmaxBackward>)

In [20]:
max_pos = {}

In [18]:
float(out[:,1].max())

0.5132220983505249

In [21]:
max_pos[float(out[:,1].max())] = sample['post'][int(out[:,1].argmax())]

In [22]:
max_pos

{0.5132220983505249: 'AMC YOLO LFG!'}

In [2]:
run_inference = RunInference(stock_id='BB')
inference_output = run_inference.evaluate()

post_probs = inference_output['avg_post_probs']

comment_probs = inference_output['avg_comment_probs']

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

connection established
[(datetime.datetime(2021, 6, 14, 17, 1, 30),)]


Determining Sentiment From Posts:   0%|          | 0/1 [00:00<?, ?it/s]

connection established
[(datetime.datetime(2021, 6, 14, 17, 1, 32),)]
connection established
[(datetime.datetime(2021, 6, 14, 17, 1, 33),)]


/Users/ianleefmans/opt/anaconda3/envs/StockSentiment/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


connection established
[(datetime.datetime(2021, 6, 14, 17, 1, 33),)]
connection established
[(datetime.datetime(2021, 6, 14, 17, 1, 34),)]
connection established
[(datetime.datetime(2021, 6, 14, 17, 1, 34),)]
connection established
[(datetime.datetime(2021, 6, 14, 17, 1, 35),)]
connection established
[(datetime.datetime(2021, 6, 14, 17, 1, 35),)]
connection established
[(datetime.datetime(2021, 6, 14, 17, 1, 36),)]
connection established
[(datetime.datetime(2021, 6, 14, 17, 1, 36),)]
connection established
[(datetime.datetime(2021, 6, 14, 17, 1, 37),)]
connection established
[(datetime.datetime(2021, 6, 14, 17, 1, 37),)]


Determining Sentiment From Comments: 100%|██████████| 2/2 [01:39<00:00, 49.56s/it]
